# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位生物领域的专家。
你知道很多生物相关的知识。
当别人问你生物相关的问题时，你的回答总是很有趣，能轻松引起大家对生物知识的兴趣。

这是一个问题：
{input}"""


computerist_template = """你是一位资深的计算机老师。
你时刻关注着计算机领域的前沿知识，与时俱进。
当别人问你计算机相关的问题时，你总能给出正确的回答。

这是一个问题：
{input}"""


chineseLinguist_template = """你是一位非常著名的汉语言文学老师。
你在汉语言文学领域有着非常卓越的成就。
当别人问你汉语言相关的问题时，你总能侃侃而谈。

这是一个问题：
{input}"""



In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computerist_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chineseLinguist_template,
    }
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", )

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

D:\software\miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
D:\software\miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [11]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [14]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n\n黑体辐射是指一个完美吸收和辐射的物体在一定温度下发出的电磁波。它的频率和强度取决于物体的温度，而与物体的材料和形状无关。黑体辐射对于理解热辐射和量子力学的发展非常重要。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40 such that adding one to this prime number is divisible by 3?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40 such that adding one to this prime number is divisible by 3?', 'text': '\n\n首先，我们需要找到大于40的第一个质数。质数是只能被1和自身整除的数，所以我们可以逐个检查大于40的数是否为质数。通过计算，我们可以得知41是大于40的第一个质数。\n\n接下来，我们需要找到一个数，使得当我们将41与它相加后，结果能被3整除。我们可以通过试错的方法来找到这个数。首先，我们可以找到41的倍数，例如82、123、164等等。但是，这些数与41相加后的结果都不能被3整除。接着，我们可以尝试将41与3相乘，得到123。我们发现，41加上123的结果为164，可以被3整除。\n\n因此，答案为164。164加上1后为165，可以被3整除，同时164也是一个质数。所以，164是第一个大于40的质数，且加上'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度的天体，它的引力非常强大，甚至连光也无法逃脱。它是由星体在死亡后因为自身质量过大而发生坍缩形成的。它的存在是基于爱因斯坦的广义相对论，而目前仍然是物理学中令人困惑的领域之一。虽然我们无法直接观测黑洞，但通过观测它们周围的物质运动和辐射，我们可以推断它们的存在。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [19]:
print(chain.invoke("母亲是O型血，父亲血型未知，生下的女儿可能是什么血型？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '母亲是O型血，父亲血型未知，生下的女儿可能是什么血型？'}
> Finished chain.
{'input': '母亲是O型血，父亲血型未知，生下的女儿可能是什么血型？', 'text': '\n\n根据遗传学知识，女儿的血型可能有三种情况：O型、A型或B型。因为母亲是O型血，她只能传递O型血型的基因给女儿。父亲的血型未知，所以无法确定他可以传递哪种血型的基因给女儿。如果父亲也是O型血，那么女儿就有50%的可能性是O型血，25%的可能性是A型血，25%的可能性是B型血。如果父亲是A型或B型血，女儿就有50%的可能性是A型血或B型血。因此，女儿可能是O型、A型或B型血型中的任意一种。当然，也有极小的概率存在女儿是AB型血的情况，但这种情况'}


In [20]:
print(chain.invoke("不同的电脑可能会有相同的IP地址吗？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '不同的电脑可能会有相同的IP地址吗？'}
> Finished chain.
{'input': '不同的电脑可能会有相同的IP地址吗？', 'text': '\n\n不同的电脑在同一网络环境下是不可能拥有相同的IP地址的，因为IP地址是用来唯一标识一个网络设备的，如果存在两个相同的IP地址，那么网络就无法正确地将数据传输到目标设备。所以，不同的电脑不可能拥有相同的IP地址。'}


In [21]:
print(chain.invoke("红楼梦为什么是名著？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '红楼梦为什么是名著？'}
> Finished chain.
{'input': '红楼梦为什么是名著？', 'text': '\n\n红楼梦被认为是一部名著，主要是因为它在文学形式、社会背景、人物刻画等方面都具有非常高的艺术价值。首先，它是一部长篇小说，结构严谨，情节曲折，具有很强的叙事能力。其次，它描绘了清朝封建社会的生活，展现了当时社会阶层、家族关系、婚姻制度等方面的特点，具有很强的历史价值。再者，小说中的人物形象栩栩如生，个性鲜明，塑造了一批经典的文学形象，具有很强的艺术魅力。总的来说，红楼梦具有丰富的文学内涵，从多个角度展'}
